In [5]:
import re
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles, MurckoScaffoldSmilesFromSmiles
from rdkit.DataStructs.cDataStructs import ExplicitBitVect, TanimotoSimilarity
import chromadb
from chromadb import Client, Settings, EmbeddingFunction

In [ ]:
sample_of_chunk = 'The structure in SMILES is SMILES{CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[C@H](C)C(=O)N(C)[C@@H](C)C(=O)N(C)[C@@H](Cc2ccccc2)C(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H]([C@@H](C)O)C(=O)N(C)CC(=O)N1C}, in HELM representation is PEPTIDE181{[dA].[meA].[meF].[meL].T.[Sar].[meL].[Ser(tBu)].[meL].D.[-pip]}$PEPTIDE181,PEPTIDE181,1:R1-10:R3$$$, permeability value is -4.37'

In [2]:
data_path = '../data/CycPeptMPDB_Peptide_Assay_PAMPA.csv'

In [3]:
radius=4
nBits=512

In [ ]:
smiles = 'CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[C@H](C)C(=O)N(C)[C@@H](C)C(=O)N(C)[C@@H](Cc2ccccc2)C(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H]([C@@H](C)O)C(=O)N(C)CC(=O)N1C'

In [ ]:
def get_morgan_fingerprint(smiles, radius=2, nBits=512):
    mol = Chem.MolFromSmiles(smiles)
    morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
    arr = np.zeros((1,), dtype=np.int8)
    ConvertToNumpyArray(morgan_fp, arr)
    return arr.tolist()


def find_smiles(text):
    pattern = r'SMILES{([^}]*)}'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None
    
def chemical_embedding(text):
    return get_morgan_fingerprint(find_smiles(text), radius, nBits)

In [6]:


class ChemicalEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input):
        smiles = self.find_smiles(input)
        if smiles:
            return self.get_morgan_fingerprint(smiles)
        return None

    def find_smiles(self, text):
        pattern = r'SMILES{([^}]*)}'
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        else:
            return None

    def get_morgan_fingerprint(self, smiles, radius=2, nBits=512):
        mol = Chem.MolFromSmiles(smiles)
        morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
        arr = np.zeros((1,), dtype=np.int8)
        ConvertToNumpyArray(morgan_fp, arr)
        return arr.tolist()
    
    @staticmethod
    def list_to_morgan_fp(self, bit_list):
        """Convert a list of bits back to an RDKit ExplicitBitVect object."""
        nBits=len(bit_list)
        bit_vector = ExplicitBitVect(nBits)
        for i, bit in enumerate(bit_list):
            if bit:
                bit_vector.SetBit(i)
        return bit_vector


In [7]:
def split_dataset_by_scaffold_and_similarity(df):
    '''Based on scaffold and stratification: it takes the least similar scaffold and put aside in validation set, 
        the rest split with taking into account distribution of scaffolds, the single instances goes to train'''
    
    df['Scaffold'] = df['SMILES'].apply(MurckoScaffoldSmilesFromSmiles)
    fingerprints = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi), radius=2, nBits=1024) for smi in df["SMILES"]]

    mean_tan_sim = []

    for i, fp in enumerate(fingerprints):
        similarities = DataStructs.BulkTanimotoSimilarity(fp, fingerprints[:i] + fingerprints[i+1:])
        mean_similarity = sum(similarities) / len(similarities)
        mean_tan_sim.append(mean_similarity)

    df['Mean_Tanimoto_Similarity'] = mean_tan_sim

    df_sorted = df.sort_values(by='Mean_Tanimoto_Similarity')
    validation_set = df_sorted.iloc[:int(len(df) * 0.1)]

    df_remaining = df_sorted.iloc[int(len(df) * 0.1):]

    scaffold_counts = df_remaining['Scaffold'].value_counts()
    single_instance_scaffolds = scaffold_counts[scaffold_counts == 1].index
    multi_instance_df = df_remaining[~df_remaining['Scaffold'].isin(single_instance_scaffolds)]
    single_instance_df = df_remaining[df_remaining['Scaffold'].isin(single_instance_scaffolds)]

    train_size = 0.8 / (0.8 + 0.1) 

    train_set, test_set = train_test_split(multi_instance_df, 
                                       test_size=1-train_size, 
                                       stratify=multi_instance_df['Scaffold'],
                                       random_state=42)


    train_set = pd.concat([train_set, single_instance_df])
    return train_set, test_set, validation_set



In [8]:
def convert_aldehydes_to_acids(smiles_list):
    rxn_smarts = '[CX3H1:1](=O)[H].[OH2:2]>>[CX3:1](=O)[O:2]'
    rxn = AllChem.ReactionFromSmarts(rxn_smarts)
    
    acid_smiles_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        ps = rxn.RunReactants((mol, Chem.MolFromSmiles('O')))
        if ps:
            product_mol = ps[0][0]  
            Chem.SanitizeMol(product_mol)  
            acid_smiles = Chem.MolToSmiles(Chem.RemoveHs(product_mol), isomericSmiles=True, canonical=True)
            acid_smiles_list.append(acid_smiles)
        else:
            print('Issue with converting C=O into -COOH')
            acid_smiles_list.append(smiles)
    
    return acid_smiles_list

def tokenize_peptides(smiles):
    mol = Chem.MolFromSmiles(smiles)
    pat = Chem.MolFromSmarts('NC=O')
    
    # Finding the largest ring
    def find_largest_ring(mol):
        sssr = Chem.GetSymmSSSR(mol)
        largest_ring = max(sssr, key=len)
        return set(largest_ring)

    largest_ring = find_largest_ring(mol)
    matches = mol.GetSubstructMatches(pat)
    
    emol = Chem.EditableMol(mol)

    bonds_to_break = []

    for match in matches:
        N_idx, C_idx, O_idx = match
        if N_idx in largest_ring and C_idx in largest_ring:
            bonds_to_break.append((N_idx, C_idx))

    # Break bonds 
    for N_idx, C_idx in sorted(bonds_to_break, reverse=True):  # Sort and reverse to avoid indexing issues
        emol.RemoveBond(N_idx, C_idx)

    fragmented_mol = emol.GetMol()
    frags = Chem.GetMolFrags(fragmented_mol, asMols=True, sanitizeFrags=True)
    fragment_smiles = [Chem.MolToSmiles(frag) for frag in frags]

    return convert_aldehydes_to_acids(fragment_smiles)

In [9]:
def format_row(row):
    return f"The structure in SMILES is SMILES{{{row['SMILES']}}}, contains following amino acids {row['tokens']}, permeability value is {row['PAMPA']}"


In [10]:
def format_row_test(row):
    return f"The structure in SMILES is SMILES{{{row['SMILES']}}}, contains following amino acids {row['tokens']}"

In [11]:
df = pd.read_csv(data_path)
df['tokens'] = df['SMILES'].apply(tokenize_peptides)
train_set, test_set, validation_set = split_dataset_by_scaffold_and_similarity(df)
train_set['Formatted_String'] = train_set.apply(format_row, axis=1)
test_set['Formatted_String'] = test_set.apply(format_row_test, axis=1)
validation_set['Formatted_String'] = validation_set.apply(format_row_test, axis=1)

/tmp/ipykernel_21178/2955250583.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [ ]:
# def tanimoto_similarity(fp1, fp2):
#     bv1 = list_to_morgan_fp(fp1)
#     bv2 = list_to_morgan_fp(fp2)
#     return TanimotoSimilarity(bv1, bv2)

# list_of_emb = []
# for molecule in train_set['Formatted_String'].head(2):
#     list_of_emb.append(chemical_embedding(molecule))

# tanimoto_similarity(list_of_emb[0], list_of_emb[1])

In [17]:

class ChemicalEmbeddingGenerator:
    @staticmethod
    def generate_embedding(smiles_string):
        try:
            mol = Chem.MolFromSmiles(smiles_string)
            if mol is not None:
                morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=512)
                arr = np.zeros((1,), dtype=np.int8)
                ConvertToNumpyArray(morgan_fp, arr)
                print("Embedding generated successfully.")
                return arr.tolist()
            else:
                print("Invalid SMILES string.")
                return None
        except Exception as e:
            print(f"An error occurred while generating embedding: {e}")
            return None

class DataFrameManager:
    def __init__(self, save_path):
        self.embedding_generator = ChemicalEmbeddingGenerator()
        self.chroma_client = chromadb.chromadb.PersistentClient(path=save_path)
        self.collection_name = "chemical_data_collection"
        self.collection = self.chroma_client.get_or_create_collection(name=self.collection_name,
                                                                      metadata={"hnsw:space": "cosine"})
        
    def add_texts_to_collection(self, dataframe, text_column, columns_to_keep):
        embeddings_list = []
        documents_list = []
        metadatas_list = []
        ids_list = []
        
        for index, row in dataframe.iterrows():
            smiles = self.extract_smiles(row[text_column])
            if smiles:
                embedding = self.embedding_generator.generate_embedding(smiles)
                if embedding is not None:
                    unique_id = f"row_{index}"
                    embeddings_list.append(embedding)
                    documents_list.append(row[text_column])

                    metadata = {column: row[column] for column in columns_to_keep if row[column] is not None}
                    
                    metadatas_list.append(metadata)
                    ids_list.append(unique_id)
        
        self.collection.add(
            embeddings=embeddings_list,
            documents=documents_list,
            metadatas=metadatas_list,
            ids=ids_list
        )
        print("Data added to collection successfully.")
        return self.collection

    @staticmethod
    def extract_smiles(text):
        pattern = r'SMILES{([^}]*)}'
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        return None

save_path = './'
manager = DataFrameManager(save_path=save_path)
manager.add_texts_to_collection(train_set, 'Formatted_String', ['Formatted_String'])


OperationalError: attempt to write a readonly database

In [16]:
manager.collection.get(include=[ "embeddings"])

{'ids': ['row_1',
  'row_10',
  'row_100',
  'row_1000',
  'row_1002',
  'row_1003',
  'row_1004',
  'row_1006',
  'row_1007',
  'row_1009',
  'row_101',
  'row_1010',
  'row_1013',
  'row_1016',
  'row_1017',
  'row_1018',
  'row_1019',
  'row_102',
  'row_1021',
  'row_1022',
  'row_1023',
  'row_1024',
  'row_1025',
  'row_1026',
  'row_1027',
  'row_103',
  'row_1030',
  'row_1031',
  'row_1038',
  'row_1039',
  'row_104',
  'row_1043',
  'row_1045',
  'row_1046',
  'row_1047',
  'row_1048',
  'row_1049',
  'row_105',
  'row_1050',
  'row_1052',
  'row_1053',
  'row_1054',
  'row_1055',
  'row_1056',
  'row_1057',
  'row_1059',
  'row_106',
  'row_1060',
  'row_1061',
  'row_1062',
  'row_1063',
  'row_1064',
  'row_1065',
  'row_1066',
  'row_1067',
  'row_1068',
  'row_1069',
  'row_107',
  'row_1070',
  'row_1071',
  'row_1072',
  'row_1073',
  'row_1074',
  'row_1075',
  'row_1077',
  'row_1079',
  'row_1080',
  'row_1081',
  'row_1082',
  'row_1083',
  'row_1084',
  'row_1085'

In [13]:
import chromadb

# Initialize the ChromaDB client
client = chromadb.Client(chromadb.Settings())

# Access the collection by name
collection_name = "chemical_data_collection"
collection = client.get_collection(name=collection_name)


peeked_results = collection.peek(limit=1)

print(peeked_results)

ValueError: Collection chemical_data_collection does not exist.

In [ ]:

# Example molecules
mol1 = Chem.MolFromSmiles('CCO')
mol2 = Chem.MolFromSmiles('CCCO')

# Generate fingerprints
fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, radius=2, nBits=512)
fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, radius=2, nBits=512)

# Compute Tanimoto similarity
similarity = DataStructs.TanimotoSimilarity(fp1, fp2)
print(f"Tanimoto Similarity: {similarity}")

In [ ]:
# import pandas as pd
# from rdkit import Chem
# from rdkit.Chem import AllChem, DataStructs
# import numpy as np

# def get_fingerprints(smiles_list):
#     """ Generate fingerprints for a list of SMILES strings. """
#     return [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smile), radius=2, nBits=1024) for smile in smiles_list]

# def calculate_tanimoto(fps, target_fp):
#     """ Calculate Tanimoto similarity between a target fingerprint and a list of fingerprints. """
#     return [DataStructs.TanimotoSimilarity(target_fp, fp) for fp in fps]

# def find_similar_molecules(df, top_n=30):
#     """ Pick a random molecule and find the top N similar molecules based on Tanimoto similarity. """
#     # Randomly select a molecule
#     train_set, test_set, validation_set = split_dataset_by_scaffold_and_similarity(df)
#     index_of_min = test_set['Mean_Tanimoto_Similarity'].idxmin()
#     random_row = test_set.loc[[index_of_min]]
#     #random_row = validation_set.sample(n=1)
#     random_smile = random_row['SMILES'].iloc[0]
#     random_mol_fp = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(random_smile), radius=2, nBits=1024)
    
#     # Get fingerprints for all molecules
#     all_fps = get_fingerprints(train_set['SMILES'])
    
#     # Calculate similarities
#     similarities = calculate_tanimoto(all_fps, random_mol_fp)
    
#     # Add similarities to the dataframe
#     train_set['Similarity'] = similarities
    
#     # Sort by similarity and select top N
#     top_similar = train_set.sort_values(by='Similarity', ascending=False).head(top_n + 1)  # +1 because it includes the selected molecule itself
    
#     # Optionally, you might want to exclude the selected molecule from the result
#     top_similar = top_similar[top_similar['SMILES'] != random_smile]
    
#     return random_row, top_similar

# # Example usage
# # Assuming 'df' is your DataFrame and it has a column 'SMILES' with SMILES strings
# random_molecule, similar_molecules = find_similar_molecules(df, top_n=30)
